# Hoja de Trabajo 2

Gabriel García - 21352        
Luis Montenegro - 21699

# Ejercicio 1 - Experimentación Practica

En esta actividad, implementará y comparará diferentes funciones de pérdida y técnicas de regularización utilizando PyTorch. Utilizará el conjunto de datos de Iris para una tarea de clasificación y una arquitectura básica de red neuronal de feedforward. El objetivo es observar cómo las diferentes opciones impactan la convergencia y el rendimiento del modelo.

In [19]:
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
import numpy as np

# Task 1 - Preparación del conjunto de datos

Cargue el conjunto de datos de Iris utilizando bibliotecas como sklearn.datasets. Luego, divida el conjunto de datos en conjuntos de entrenamiento y validación

In [20]:
# Cargar el dataset de iris
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [21]:
# Escalar los datos
scaler = StandardScaler()
X = scaler.fit_transform(x)

In [22]:
# Dividir el conjunto de datos en conjuntos de entrenamiento y validación (80% entrenamiento, 20% validación)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [23]:
# Convertir los datos a tensores de PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

In [24]:
# Crear conjuntos de datos y cargadores de datos
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [25]:
# Imprimir las formas de los conjuntos de datos para verificación
print(f"Tamaño del conjunto de entrenamiento: {len(train_loader.dataset)} muestras")
print(f"Tamaño del conjunto de validación: {len(val_loader.dataset)} muestras")

Tamaño del conjunto de entrenamiento: 120 muestras
Tamaño del conjunto de validación: 30 muestras


# Task 2 - Arquitectura modelo
Cree una red neuronal feedforward simple utilizando nn.Module de PyTorch. Luego, defina capa de entrada, capas ocultas y capa de salida. Después, elija las funciones de activación y el número de neuronas por capa.

In [26]:
import torch.nn as nn
import torch.nn.functional as F

In [27]:
# Definir la arquitectura de la red neuronal
class SimpleFeedforwardNN(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(SimpleFeedforwardNN, self).__init__()
        # Definir la capa de entrada
        self.fc1 = nn.Linear(input_size, hidden_size1)
        # Definir la primera capa oculta
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        # Definir la capa de salida
        self.fc3 = nn.Linear(hidden_size2, output_size)
        
    def forward(self, x):
        # Aplicar activación ReLU a la capa de entrada
        x = F.relu(self.fc1(x))
        # Aplicar activación ReLU a la primera capa oculta
        x = F.relu(self.fc2(x))
        # Aplicar activación Softmax a la capa de salida para obtener probabilidades
        x = F.log_softmax(self.fc3(x), dim=1)
        return x

In [28]:
# Parámetros del modelo
input_size = 4  # El conjunto de datos de Iris tiene 4 características de entrada
hidden_size1 = 10  # Número de neuronas en la primera capa oculta
hidden_size2 = 8   # Número de neuronas en la segunda capa oculta
output_size = 3    # El conjunto de datos de Iris tiene 3 clases de salida

# Instanciar el modelo
model = SimpleFeedforwardNN(input_size, hidden_size1, hidden_size2, output_size)

# Mostrar la arquitectura del modelo
print(model)

SimpleFeedforwardNN(
  (fc1): Linear(in_features=4, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=3, bias=True)
)
